In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.fundaments.intt import Intt
from everest.ptolemaic.schema import Schema

class MyClass(metaclass=Schema):
    a: Intt

In [ ]:
chora = MyClass.oid.chora

In [ ]:
chora

In [ ]:
chora[0]

In [ ]:
MyClass.oid[0]